## 1.1 将序列分解为单独的变量

### 问题

我们有一个包含N个元素的元组或序列，现在想将它分解为N个单独的变量

### 解决方案

分解可迭代对象，变量的总数和结构要与序列相吻合

In [3]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
_, shares, price, _ = data
print('shares:%s ,price:%s'%(shares, price))

shares:50 ,price:91.1


## 1.2 从任意长度的可迭代对象中分解元素

假设开设了一门课程，并决定在期末的作业成绩中去掉第一个和最后一个，只对中间剩下的成绩做平均分统计

In [4]:
import random
grades = [random.randint(1, 100) for _ in range(24)]
print(grades)

[61, 2, 1, 78, 64, 8, 70, 10, 11, 86, 89, 79, 36, 99, 37, 49, 88, 28, 40, 54, 88, 6, 85, 30]


In [6]:
def drop_first_last(grades):
    first, *middle, last = grades
    return middle

middle = drop_first_last(grades)
print(middle)

[2, 1, 78, 64, 8, 70, 10, 11, 86, 89, 79, 36, 99, 37, 49, 88, 28, 40, 54, 88, 6, 85]


\*式的语法在迭代一个变长的元组序列时尤其有用

In [7]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]
def do_foo(x, y):
    print('foo', x, y)
def do_bar(s):
    print('bar', s)
for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)


foo 1 2
bar hello
foo 3 4


有时候可能想分解出某些值然后丢弃它们

In [9]:
record = ['ACME', 50, 123.45, (12, 18, 2012)]
name, *_, (*_, year) = record
print(name, year)

ACME 2012


## 1.3 保存最后N个元素

### 问题

我们希望在迭代或是其他形式的处理过程中对最后几项记录做一个有限的历史记录统计

### 解决方案

`collections.deque`

下面的代码对一系列文本做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本

In [13]:
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines
        previous_lines.append(line)
    
# Example use on a file
with open(r'C:\Users\Ph\Desktop\log\手机号解绑.txt') as f:
    for line, prevlines in search(f, '网易云音乐', 5):
        for pline in prevlines:
            print(pline, end='')
        print(line, end='')
        print('-'*24)

摩拜单车
西电教务处
bilibili      # 很重要
简书
36氪
网易云音乐
------------------------


`deque(maxlen=N)`创建了一个固定长度的队列。当有新纪录加入而队列已满时会自动移除最老的那条记录

In [14]:
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
q.append(4)
print(q)

deque([2, 3, 4], maxlen=3)


从队列两端添加或弹出元素的复杂度都是O(1)。当从列表的头部插入或移除元素时，列表的复杂度为O(N)。

In [15]:
q = deque()
q.append(1)
q.append(2)
q.append(3)
print(q)
q.appendleft(4)
print(q)
p = q.pop()
print(p)
print(q)
q.popleft()
print(q)

deque([1, 2, 3])
deque([4, 1, 2, 3])
3
deque([4, 1, 2])
deque([1, 2])


## 1.4 找到最大或最小的N个元素

### 问题

我们想在某个集合中找到最大或最小的N个元素

### 解决方案

heapq模块中有两个函数 —— nlargest() 和 nsmallest() —— 它们正是我们所需要的

In [16]:
import heapq
import random

nums = [random.randint(1, 40) for _ in range(15)]
print(nums)
print(heapq.nlargest(4, nums))
print(heapq.nsmallest(4, nums))

[1, 15, 14, 12, 37, 14, 10, 35, 13, 6, 16, 16, 5, 31, 22]
[37, 35, 31, 22]
[1, 5, 6, 10]


这两个函数都可以接受一个参数key，从而允许它们工作在更加复杂的数据结构之上

In [17]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price': 91.1},
    {'name': 'AAPL', 'shares': 50, 'price': 543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares':45, 'price': 16.35},
    {'name': 'ACME', 'shares':75, 'price': 115.65},
]

cheap = heapq.nsmallest(3, portfolio, key=lambda s:s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s:s['price'])
print(f'cheap: {cheap}')
print(f'expensive: {expensive}')

cheap: [{'name': 'YHOO', 'shares': 45, 'price': 16.35}, {'name': 'FB', 'shares': 200, 'price': 21.09}, {'name': 'HPQ', 'shares': 35, 'price': 31.75}]
expensive: [{'name': 'AAPL', 'shares': 50, 'price': 543.22}, {'name': 'ACME', 'shares': 75, 'price': 115.65}, {'name': 'IBM', 'shares': 100, 'price': 91.1}]


## 1.5 实现优先级队列

### 问题

我们想要实现一个队列，它能够以给定的优先级来对元素排序，且每次 pop 操作时都会返回优先级最高的那个元素

### 解决方案

利用 heapq 模块实现简单的优先级队列

In [20]:
import heapq
class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
    
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))  # 将元组推入 self._queue
        self._index += 1
    
    def pop(self):
        return heapq.heappop(self._queue)[-1]  # 将 self._queue[0] 最小的元素 self._queue[-1] 弹出

class Item:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)

In [24]:
q = PriorityQueue()
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)
print(q.pop())
print(q.pop())
print(q.pop())
print(q.pop())

Item('bar')
Item('spam')
Item('foo')
Item('grok')


In [26]:
Item('foo') > Item('bar')

TypeError: '>' not supported between instances of 'Item' and 'Item'

In [28]:
a = (1, Item('foo'))
b = (5, Item('bar'))
c = (1, Item('grok'))
print('a < b is {}'.format(a < b))
print('a < c is {}'.format(a < c))

a < b is True


TypeError: '<' not supported between instances of 'Item' and 'Item'

In [29]:
a = (1, 0, Item('foo'))
b = (5, 1, Item('bar'))
c = (1, 2, Item('grok'))
print('a < b is {}'.format(a < b))
print('a < c is {}'.format(a < c))

a < b is True
a < c is True


## 1.6 在字典中将键映射到多个值上

### 问题

我们想要一个能将键（key）映射到多个值的字典（即所谓的一键多值字典[multidict]）

### 解决方案

利用 collections 模块的 defaultdict

In [32]:
from collections import defaultdict

# defaultdict 会自动初始化第一个值，会自动创建字典表项以待稍后的访问
# 通俗的说就是 defaultdict 访问没初始化过的 Key 不会报 KeyError，而是返回空的 list or set
dl = defaultdict(list)
dl['a'].append(1)
dl['a'].append(2)
dl['b'].append(4)
dl['b'].append(8)
print(dl)
ds = defaultdict(set)
ds['a'].add(1)
ds['a'].add(2)
ds['b'].add(4)
ds['b'].add(8)
print(ds)

defaultdict(<class 'list'>, {'a': [1, 2], 'b': [4, 8]})
defaultdict(<class 'set'>, {'a': {1, 2}, 'b': {8, 4}})


## 1.7 让字典保持有序

### 问题

我们想创建一个字典，同时当对字典做迭代或序列化操作时，也能控制其中元素的顺序

### 解决方案

利用 collections 模块的 OrderedDict

### 讨论

OrderedDict 内部维护了一个双向链表，它会根据元素加入的顺序来排列键的位置。
第一个新加入的元素被放置在链表的末尾。接下来对已存在的键做重新赋值不会改变键的顺序。

warning：OrderedDict的大小是普通字典的2倍多

In [33]:
from time import time
from random import randint
from collections import OrderedDict

d = OrderedDict()
players = list('ABCDEFGH')
start = time()

for i in range(8):
    input()
    p = players.pop(randint(0, 7 - i))
    end = time()
    print(i + 1, p, end-start)
    d[p] = (i+1, end - start)

print('\n', '-' * 24, '\n')
for k in d:
    print(k, d[k])


1 H 6.950457334518433

2 C 7.534353733062744

3 E 8.800331354141235

4 D 10.574729442596436

5 A 12.161805152893066

6 B 12.522283792495728

7 F 13.620580673217773

8 G 15.475238800048828

 ------------------------ 

H (1, 6.950457334518433)
C (2, 7.534353733062744)
E (3, 8.800331354141235)
D (4, 10.574729442596436)
A (5, 12.161805152893066)
B (6, 12.522283792495728)
F (7, 13.620580673217773)
G (8, 15.475238800048828)


## 1.8 与字典有关的计算问题

### 问题

我们想在字典上对数据执行各式各样的计算（比如求最小值、最大值、排序等）

### 解决方案

利用 zip() 将字典的 key 和 value 反过来

In [34]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}
min_price = min(zip(prices.values(), prices.keys()))
print(f'min_price is {min_price}')
max_price = max(zip(prices.values(), prices.keys()))
print(f'max_price is {max_price}')
prices_sorted = sorted(zip(prices.values(), prices.keys()))
print('*'*24)
print(prices_sorted)

min_price is (10.75, 'FB')
max_price is (612.78, 'AAPL')
************************
[(10.75, 'FB'), (37.2, 'HPQ'), (45.23, 'ACME'), (205.55, 'IBM'), (612.78, 'AAPL')]


## 1.9 在两个字典中寻找相同点

### 问题

有两个字典，我们想找出它们中间可能相同的地方（相同的键、相同的值等）

### 解决方案

通过 keys() 或者 items() 方法执行常见的集合操作

In [35]:
a = {
    'x': 1,
    'y': 2,
    'z': 3,
}
b = {
    'w': 10,
    'x': 11,
    'y': 2,
}

### 讨论

字典的 keys() 会返回 keys-view 对象，支持常见的集合操作，
字典的 items() 会返回 items-view 对象，也支持常见的集合操作

但是 values() 方法并不支持集合操作，但是可以先将值转化为集合来实现

In [36]:
a.keys() & b.keys()

{'x', 'y'}

In [37]:
a.keys() - b.keys()

{'z'}

In [38]:
a.items() & b.items()

{('y', 2)}

## 1.10 从序列中移除重复项且保持元素间顺序不变

### 问题

我们想去除序列中出现的重复元素，但仍然保持剩下的元素顺序不变

### 解决方案

如果序列中的值是可哈希（hashable）的，使用集合和生成器

可哈希：对象在生存期内必须不可变，需要有一个 __hash__() 方法

In [44]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [46]:
import random
a = [random.randint(1, 10) for _ in range(1, 16)]
print(a)
print(list(dedupe(a)))

[3, 4, 10, 7, 2, 10, 7, 7, 4, 1, 9, 1, 7, 8, 6]
[3, 4, 10, 7, 2, 1, 9, 8, 6]


如果序列中的元素是**不可哈希**的对象

In [47]:
def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)

In [48]:
a = [{'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]
print(list(dedupe(a, key=lambda d: (d['x'], d['y']))))
print(list(dedupe(a, key=lambda d: (d['x']))))

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]
[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]


## 1.11 对切片命名

### 问题

我们的代码已经变得无法阅读，到处都是硬编码的切片索引，我们想将它们清理干净

In [50]:
items = [0, 1, 2, 3, 4, 5, 6]
print(items)
a = slice(2, 4)
print(items[2:4])
print(items[a])
items[a] = [10, 11]
print(items)
del items[a]
print(items)

[0, 1, 2, 3, 4, 5, 6]
[2, 3]
[2, 3]
[0, 1, 10, 11, 4, 5, 6]
[0, 1, 4, 5, 6]


In [51]:
a = slice(10, 50, 2)
print(a.start)
print(a.stop)
print(a.step)

10
50
2


使用 indices(size) 方法将切片映射到特定大小的序列上  可避免出现 IndexError 异常

In [53]:
s = 'HelloWorld'
a = slice(5, 20, 2)
a.indices(len(s))

(5, 10, 2)

In [54]:
for i in range(*a.indices(len(s))):
    print(s[i])

W
r
d


## 1.12 找出序列中出现次数最多的元素

### 问题

我们有一个元素序列，想知道在序列中出现次数最多的元素是什么

### 解决方案

collections 模块中的 Counter 类正是为此类问题所设计的

In [58]:
from random import randint

data = [randint(0, 10) for _ in range(20)]
print(data)

[10, 2, 0, 3, 3, 0, 8, 9, 10, 1, 8, 2, 0, 9, 2, 3, 3, 9, 1, 2]


In [59]:
from collections import Counter

c1 = Counter(data)
print(c1, '\n')

print(c1.most_common(3))
print(c1.most_common(1)[0][0])

Counter({2: 4, 3: 4, 0: 3, 9: 3, 10: 2, 8: 2, 1: 2}) 

[(2, 4), (3, 4), (0, 3)]
2


In [57]:
from collections import Counter

c2 = Counter(data)
print(c2, '\n')

print(c2.most_common(3))
print(c2.most_common(1)[0][0])

Counter({7: 4, 8: 3, 0: 3, 2: 2, 9: 2, 10: 2, 3: 2, 5: 1, 6: 1}) 

[(7, 4), (8, 3), (0, 3)]
7


In [62]:
a = c1 + c2
print(a)
b = c1 - c2
print(b)

Counter({2: 6, 0: 6, 3: 6, 8: 5, 9: 5, 10: 4, 7: 4, 1: 2, 5: 1, 6: 1})
Counter({2: 2, 3: 2, 1: 2, 9: 1})


## 1.13 通过公共键对字典列表排序

### 问题

我们有一个字典列表，想根据一个或多个字典中的值来对列表进行排序

### 解决方案

利用 operator 模块中的 itemgetter 函数

In [63]:
rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
]

In [66]:
from operator import itemgetter
from pprint import pprint
rows_by_fname = sorted(rows, key=itemgetter('fname'))
rows_by_uid = sorted(rows, key=itemgetter('uid'))

pprint(rows_by_fname)
print()
pprint(rows_by_uid)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]

[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


itemgetter()函数还可以接受多个键

In [67]:
rows_by_lfname = sorted(rows, key=itemgetter('lname', 'fname'))
pprint(rows_by_lfname)

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
 {'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]


有时候会用 lambda 表达式来取代 itemgetter() 的功能，但是 itemgetter() 通常会运行得更快一些

In [69]:
rows_by_fname = sorted(rows, key=lambda r: r['fname'])
rows_by_lfname = sorted(rows, key=lambda r: (r['lname'], r['fname']))
pprint(rows_by_fname)
print()
pprint(rows_by_lfname)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
 {'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]


此操作同样适用于 min() 和 max() 这样的函数

In [70]:
print('min: ', min(rows, key=itemgetter('uid')))
print('max: ', max(rows, key=itemgetter('uid')))

min:  {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}
max:  {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}


## 1.14 对不原生支持比较操作的对象排序

### 问题

我们想在同一个类的实例之间做排序，但是它们并不原生支持比较操作

### 解决方案

内建的 sorted() 函数可接受一个用来传递可调用对象（callable）的参数 key，而该可调用对象会返回待排序对象中的某些值，sorted 则利用这些值来比较对象。

In [73]:
from operator import attrgetter

class User:
    def __init__(self, user_id):
        self.user_id = user_id
    def __repr__(self):
        return 'User({})'.format(self.user_id)
users = [User(23), User(3), User(99)]
print(users)
print(sorted(users, key=lambda u:u.user_id))  # 方法一
print(sorted(users, key=attrgetter('user_id')))  # 方法二

[User(23), User(3), User(99)]
[User(3), User(23), User(99)]
[User(3), User(23), User(99)]


In [74]:
print('min: ', min(users, key=lambda u:u.user_id))
print('max: ', max(users, key=lambda u:u.user_id))

min:  User(3)
max:  User(99)


## 1.15 根据字段将记录分组

### 问题

有一系列的字典或对象实例，我们想根据某个特定的字段（比如说日期）来分组迭代数据

### 解决方案

itertools.groupby()函数在对数据进行分组时特别有用

In [75]:
rows = [
    {'address': '5412 N CLARK', 'date': '07/01/2012'},
    {'address': '5148 N CLARK', 'date': '07/04/2012'},
    {'address': '5800 E 58TH', 'date': '07/02/2012'},
    {'address': '2122 N CLARK', 'date': '07/03/2012'},
    {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
    {'address': '1060 W ADDISON', 'date': '07/02/2012'},
    {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
    {'address': '1039 W GRANVILLE', 'date': '07/04/2012'},
]

In [76]:
from operator import itemgetter
from itertools import groupby

order_rows = sorted(rows, key=itemgetter('date'))
print('order rows: ', order_rows)
for date, items in groupby(order_rows, key=itemgetter('date')):
    print(date)
    for i in items:
        print(' ', i)

order rows:  [{'address': '5412 N CLARK', 'date': '07/01/2012'}, {'address': '4801 N BROADWAY', 'date': '07/01/2012'}, {'address': '5800 E 58TH', 'date': '07/02/2012'}, {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}, {'address': '1060 W ADDISON', 'date': '07/02/2012'}, {'address': '2122 N CLARK', 'date': '07/03/2012'}, {'address': '5148 N CLARK', 'date': '07/04/2012'}, {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}]
07/01/2012
  {'address': '5412 N CLARK', 'date': '07/01/2012'}
  {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
  {'address': '5800 E 58TH', 'date': '07/02/2012'}
  {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
  {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
  {'address': '5148 N CLARK', 'date': '07/04/2012'}
  {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


如果只是简单地根据日期将数据分组到一起，放进一个大的数据结构中以允许进行随机访问，可以利用 defaultdict() 构建一个一键多值字典

In [78]:
from collections import defaultdict
from pprint import pprint
rows_by_date = defaultdict(list)
for row in rows:
    rows_by_date[row['date']].append(row)
pprint(rows_by_date)

defaultdict(<class 'list'>,
            {'07/01/2012': [{'address': '5412 N CLARK', 'date': '07/01/2012'},
                            {'address': '4801 N BROADWAY',
                             'date': '07/01/2012'}],
             '07/02/2012': [{'address': '5800 E 58TH', 'date': '07/02/2012'},
                            {'address': '5645 N RAVENSWOOD',
                             'date': '07/02/2012'},
                            {'address': '1060 W ADDISON',
                             'date': '07/02/2012'}],
             '07/03/2012': [{'address': '2122 N CLARK', 'date': '07/03/2012'}],
             '07/04/2012': [{'address': '5148 N CLARK', 'date': '07/04/2012'},
                            {'address': '1039 W GRANVILLE',
                             'date': '07/04/2012'}]})


## 1.16 筛选序列中的元素

### 问题

序列中有一些数据，我们需要提取出其中的值或根据某些标准对序列做删减

### 解决方案

列表推导式

In [1]:
import random
mylist = [random.randint(-10, 10) for _ in range(10)]
print(mylist)

[8, -3, -8, -6, 0, -7, -5, -3, 3, 1]


In [2]:
[n for n in mylist if n > 0]

[8, 3, 1]

In [3]:
[n for n in mylist if n < 0]

[-3, -8, -6, -7, -5, -3]

In [5]:
pos = (n for n in mylist if n < 0)
pos

<generator object <genexpr> at 0x0000018159982228>

In [6]:
for x in pos:
    print(x)

-3
-8
-6
-7
-5
-3


将处理筛选逻辑的代码放到单独的函数中，然后使用内建的 filter() 函数处理

In [7]:
values = ['1', '2', '-3', '-', '4', 'N/A', '5']

def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False
ivals = list(filter(is_int, values))  # filter() 创建了一个迭代器
print(ivals)

['1', '2', '-3', '4', '5']


In [10]:
import math

[math.sqrt(n) for n in mylist if n > 0]

[2.8284271247461903, 1.7320508075688772, 1.0]

替换掉不满足标准的值demo

In [11]:
[n if n > 0 else 0 for n in mylist]

[8, 0, 0, 0, 0, 0, 0, 0, 3, 1]

筛选工具 `itertools.compress()` ,它接受一个可迭代对象以及一个布尔选择器序列作为输入

In [12]:
from itertools import compress
addresses = [
    '5421 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK',
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GRANVILLE',
]
counts = [0, 3, 10, 4, 1, 7, 6, 1]
more5 = [n > 5 for n in counts]
print('more5: ', more5)
print(list(compress(addresses, more5)))

more5:  [False, False, True, False, False, True, True, False]
['5800 E 58TH', '1060 W ADDISON', '4801 N BROADWAY']


## 1.17 从字典中提取子集

### 问题

我们想创建一个字典，其本身是另一个字典的子集

### 解决方案

利用字典推导式（dictionary comprehension）可轻松解决

In [14]:
d = {x: random.randint(60,100) for x in range(1, 21)}
print(d)

{1: 93, 2: 61, 3: 95, 4: 87, 5: 88, 6: 99, 7: 75, 8: 68, 9: 75, 10: 80, 11: 92, 12: 97, 13: 88, 14: 64, 15: 79, 16: 93, 17: 90, 18: 86, 19: 62, 20: 65}


In [15]:
{k:v for k,v in d.items() if v >= 90}

{1: 93, 3: 95, 6: 99, 11: 92, 12: 97, 16: 93, 17: 90}

## 1.18 将名称映射到序列的元素中

### 问题

我们的代码是通过位置来访问列表或元组的，但有时候这会使代码变得有些难以阅读。实现通过名称来访问元素

### 解决方案

`collections.namedtuple()`

In [34]:
from collections import namedtuple

student = ('Jim', 16, 'male', 'jim8721@gmail.com')
Student = namedtuple('Student', ['name', 'age', 'sex', 'email'])
s = Student(*student)
print(s)
print(Student.__name__)

Student(name='Jim', age=16, sex='male', email='jim8721@gmail.com')
Student


In [21]:
print(s.name)
print(s.age)
print(s.sex)
print(s.email)

Jim
16
male
jim8721@gmail.com


In [19]:
isinstance(s, tuple)

True

### 讨论

namedtuple 是不可变的 （immutable），如果需要修改任何属性，可以通过 _replace() 方法实现

In [35]:
print(s)
s = s._replace(age=23)
print(s)

Student(name='Jim', age=16, sex='male', email='jim8721@gmail.com')
Student(name='Jim', age=23, sex='male', email='jim8721@gmail.com')


In [38]:
from collections import namedtuple
a = {'name': 'ACME', 'shares': 100, 'price': 123.45}
b = {'name': 'ACME', 'shares': 100, 'price': 123.45, 'date': '12/17/2012'}
Stock = namedtuple('Stock', ['name', 'shares', 'price', 'date', 'time'])
stock_prototype = Stock('', 0, 0.0, None, None)
def dict_to_stock(s):
    return stock_prototype._replace(**s)
print(dict_to_stock(a))
print(dict_to_stock(b))

Stock(name='ACME', shares=100, price=123.45, date=None, time=None)
Stock(name='ACME', shares=100, price=123.45, date='12/17/2012', time=None)


这里的 `stock_prototype._replace(**s)` 可以理解为将字典 s 解包成 name='ACME', shares=100, price=123.45

Demo:

In [49]:
def foo(**d):
    print(d)
foo(x='1',y=2)

{'x': '1', 'y': 2}


## 1.19 同时对数据做转换和换算

### 问题

我们需要调用一个换算（reduction）函数（例如sum（）、min（）、max（）），但首先得对数据做转换或筛选。

### 解决方案

在函数参数中使用生成器表达式。

In [59]:
import random
nums = [random.randint(1, 10) for _ in range(5)]
print(nums)
s = sum(x * x for x in nums)
print(s)

[10, 2, 8, 9, 6]
285


In [61]:
portfolio = [
    {'name': 'GOOG', 'shares': 50},
    {'name': 'YHOO', 'shares': 75},
    {'name': 'AOL', 'shares': 20},
    {'name': 'SCOX', 'shares': 65},
]
min_shares = min(s['shares'] for s in portfolio)
print(min_shares)

20


### 讨论

下面两行代码表示的是同一个意思：

In [62]:
s = sum((x * x for x in nums))
s = sum(x * x for x in nums)

如果不适用生成器表达式，可能会考虑下面这种实现：

In [63]:
s = sum([x * x for x in nums])

这也能工作，但这引入了一个额外的步骤而且创建了额外的列表。

如果nums非常巨大，那么就会创建一个庞大的临时数据结构，而且只用一次就会丢弃。

基于生成器的解决方案可以以迭代的方式转换数据，因此在内存使用上要高效得多。

In [64]:
# 使用 key 参数

min_shares = min(portfolio, key=lambda s:s['shares'])
print(min_shares)

{'name': 'AOL', 'shares': 20}


## 1.20 将多个映射合并为单个映射

### 问题

我们有多个字典或映射，想在逻辑上将它们合并为一个单独的映射结构，以此执行某些特定的操作，比如查找值或检查键是否存在

### 解决方案

利用 collections 模块的 ChainMap 类

In [70]:
from collections import ChainMap

a = {'x':1, 'z':3}
b = {'y':2, 'z':4}

c = ChainMap(a, b)
print(c['x'])  # Outputs 1 (from a)
print(c['y'])  # Outputs 2 (from b)
print(c['z'])  # Outputs 3 (from a)

1
2
3


### 讨论

如果有重复的键，这里会采用第一个映射中所对应的值。

修改映射的操作总是会作用在列出的第一个映射结构上。

In [71]:
c['z'] = 10
c['w'] = 40
del c['x']
print(a)

{'z': 10, 'w': 40}


In [72]:
del c['y']

KeyError: "Key not found in the first mapping: 'y'"

ChainMap 与带有作用域的值，比如编程语言中的变量（即全局变量、局部变量等）一起工作时特别有用

In [74]:
values = ChainMap()
values['x'] = 1
values = values.new_child()
values['x'] = 2
values = values.new_child()
values['x'] = 3
print(values)

ChainMap({'x': 3}, {'x': 2}, {'x': 1})


In [76]:
values['x']

3

In [77]:
values = values.parents
print(values['x'])
values = values.parents
print(values['x'])
print(values)

2
1
ChainMap({'x': 1})


### 合并字典

In [85]:
a = {'x':1, 'z':3}
b = {'y':2, 'z':4}

c1 = dict(a, **b)
print(c1)
c2 = dict(b, **a)
print(c2)

{'x': 1, 'z': 4, 'y': 2}
{'y': 2, 'z': 3, 'x': 1}


上面合并字典过程中后者会覆盖前者，且单独构建了一个完整的字典对象。

这么做如果其中任何一个原始字典做了修改，这个改变都不会反应到合并后的字典中。

而 ChainMap 使用的就是原始的字典，因此它不会产生这种令人不悦的行为。

In [86]:
merged = ChainMap(a, b)
print(merged['y'])
b['y'] = 5
print(merged['y'])

2
5
